In [4]:
from langchain_ollama.llms import OllamaLLM

In [5]:
llm = OllamaLLM(model="gemma3:4b")

In [6]:
res = llm.invoke("你叫什么名字？")

In [7]:
print(res)

我叫 Gemma，是由 Google DeepMind 训练的大型语言模型。


In [8]:
from langchain_core.messages import HumanMessage, SystemMessage
message = [
    SystemMessage("Translate the following from Chinese into Englshi"),
    HumanMessage("你好，你叫什么？")
]
print(message)

[SystemMessage(content='Translate the following from Chinese into Englshi', additional_kwargs={}, response_metadata={}), HumanMessage(content='你好，你叫什么？', additional_kwargs={}, response_metadata={})]


In [9]:
llm.invoke(message)

'Okay, here\'s the translation:\n\n**"Hello, what\'s your name?"** \n\nOr, more formally: **"Hello, what is your name?"** \n\nLet me know if you\'d like to translate anything else!'

In [10]:
llm.invoke("Hello")

llm.invoke([{"role": "user", "content": "Hello"}])

llm.invoke([HumanMessage("Hello")])

'Hello there! How can I help you today? 😊 \n\nDo you want to:\n\n*   Chat about something?\n*   Ask me a question?\n*   Play a game?'

In [11]:
res = llm.stream(message)
for token in res:
    print(token, end="")

Hello, what's your name? 

(Or, more literally: "Hello, you what name?")

In [12]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [20]:
prompt_mes = prompt_template.invoke({"language":"Chinese","text":"how are you"})

In [21]:
prompt_mes.to_messages()

[SystemMessage(content='Translate the following from English into Chinese', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='how are you', additional_kwargs={}, response_metadata={})]

In [22]:
response =  llm.invoke(prompt_mes)

In [23]:
response

'Here are a few ways to translate "How are you?" into Chinese, depending on the level of formality:\n\n*   **你好吗？ (Nǐ hǎo ma?)** - This is the most common and versatile translation. It\'s suitable for most situations. (你好 - nǐ hǎo - hello; 吗 - ma - a question particle)\n\n*   **最近怎么样？(Zuìjìn zěnme yàng?)** - This translates more literally to "How have you been lately?" and is good for people you haven\'t seen in a while. (最近 - zuìjìn - recently; 怎么样 - zěnme yàng - how is it?)\n\n*   **你最近好吗？(Nǐ zuìjìn hǎo ma?)** - Similar to the above, but emphasizes "recently."\n\n\n**Therefore, I recommend using 你好吗？ (Nǐ hǎo ma?) for a general "How are you?"**\n\nWould you like me to provide a response to that question in Chinese as well?'

In [24]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [26]:
documents[0].page_content

'Dogs are great companions, known for their loyalty and friendliness.'

In [27]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "HR.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

71


In [28]:
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

密级：内部
山 鹰 国 际 控 股 股 份 公 司
鹰发〔2023〕35 号
山鹰国际控股股份公司关于
印发《人力资源管理制度》（2023 年 01 版）
的通知
各单位：
现将经股份公司人力资源中心修订的《人力资源管理制
度》（2023 年 01 版）（以下简称“本制度”）印发给你们，请
遵照执行。
本制度自 2023 年 1 月 1 日起执行，原《人力资源管理
制度》（2019 年修订版）同时

{'producer': '; modified using iText® 5.5.9 ©2000-2015 iText Group NV (AGPL-version)', 'creator': 'WPS Writer', 'creationdate': '2023-01-15T11:46:25+11:46', 'author': 'llw', 'comments': '', 'company': '', 'keywords': '', 'moddate': '2023-01-15T20:27:13+08:00', 'sourcemodified': "D:20230115114625+11'46'", 'subject': '', 'title': '', 'trapped': 'False', 'qys_document_id': '3052208407203779168', 'source': 'HR.pdf', 'total_pages': 71, 'page': 0, 'page_label': '1'}


In [29]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

72

In [31]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="qwen2.5:3b")

In [34]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 2048

[0.0030309279, -0.011394628, -0.0019449119, 0.006271691, 0.018357208, -0.007348795, -0.018835261, -0.02299592, -0.01382897, 0.024247237]


In [36]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [37]:
ids = vector_store.add_documents(documents=all_splits)

In [38]:
results = vector_store.similarity_search(
    "员工离职"
)

print(results[0])

page_content='3
山鹰国际控股股份公司
人力资源管理制度
目录
第一章 总则....................................................................................... 4
第二章 管理权责............................................................................... 5
第三章 组织与岗位管理...................................................................6
第四章 招聘与录用管理...................................................................8
第五章 劳动合同管理.....................................................................16
第六章 员工考勤管理.....................................................................25
第七章 员工薪酬与福利.................................................................30
第八章 员工绩效管理.....................................................................37
第九章 员工培训与发展.................................................................39
第十章 员工异动管理.....................................................................45
第十一章 员工关系管理.................................................................56' metadata={'producer': '; modified using iText® 5.

In [40]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [41]:
ids = vector_store.add_documents(documents=all_splits)

In [42]:
results = vector_store.similarity_search(
    "员工福利有哪些？"
)

print(results[0])

page_content='3
山鹰国际控股股份公司
人力资源管理制度
目录
第一章 总则....................................................................................... 4
第二章 管理权责............................................................................... 5
第三章 组织与岗位管理...................................................................6
第四章 招聘与录用管理...................................................................8
第五章 劳动合同管理.....................................................................16
第六章 员工考勤管理.....................................................................25
第七章 员工薪酬与福利.................................................................30
第八章 员工绩效管理.....................................................................37
第九章 员工培训与发展.................................................................39
第十章 员工异动管理.....................................................................45
第十一章 员工关系管理.................................................................56' metadata={'author': 'llw', 'comments': '', 'compa

In [43]:
# Note that providers implement different scores; the score here
# is a distance metric that varies inversely with similarity.

results = vector_store.similarity_search_with_score("员工福利有哪些?")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.7716239742050995

page_content='3
山鹰国际控股股份公司
人力资源管理制度
目录
第一章 总则....................................................................................... 4
第二章 管理权责............................................................................... 5
第三章 组织与岗位管理...................................................................6
第四章 招聘与录用管理...................................................................8
第五章 劳动合同管理.....................................................................16
第六章 员工考勤管理.....................................................................25
第七章 员工薪酬与福利.................................................................30
第八章 员工绩效管理.....................................................................37
第九章 员工培训与发展.................................................................39
第十章 员工异动管理.....................................................................45
第十一章 员工关系管理.................................................................56' metadata={'author': 'l

In [46]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)

res = retriever.batch(
    [
        "新员工入职的准备材料有哪些？",
        "组织设计的基本原则是什么？",
    ],
)

In [55]:
type(res[1][1].page_content)

str

In [58]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)


class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    language: str = Field(description="The language the text is written in")


# Structured LLM
#structured_llm = llm.with_structured_output(Classification)

In [63]:
llm2 =  OllamaLLM(model = "qwen2.5:3b")
structured_llm = llm2.with_structured_output(Classification)

NotImplementedError: 